<a href="https://colab.research.google.com/github/drsunithaev/cancer_detection/blob/main/use_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use the best model saved during training

### connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### import packages

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import shutil
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
import matplotlib.patches as mpatches
from sklearn.utils import shuffle
from tqdm import tqdm

### Folder in which the pretrained model is saved

In [ ]:
my_dir = '/content/drive/MyDrive/Brain/models'

### load the model (from .pb file)

In [ ]:
my_model = tf.keras.models.load_model(my_dir)

In [ ]:
my_main_dir = '/content/drive/MyDrive/Brain'
base_dir  = my_main_dir + '/_MODELLING'
os.chdir(base_dir)

### generate training data

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR, TEST_DIR):

    # Instantiate the ImageDataGenerator class (don't forget to set the arguments to augment the images)
    train_datagen = ImageDataGenerator(rescale=1./127.5,
                                     rotation_range=30,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

    # Pass in the appropriate arguments to the flow_from_directory method
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

    # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
    valid_or_test_datagen = ImageDataGenerator(rescale=1./127.5)

    # Pass in the appropriate arguments to the flow_from_directory method
    validation_generator = valid_or_test_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='binary',
                                                                target_size=(150, 150))

    test_generator = valid_or_test_datagen.flow_from_directory(directory=TEST_DIR,
                                                                batch_size=32,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
    return train_generator, validation_generator, test_generator

In [ ]:
training_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')
testing_dir = os.path.join(base_dir, 'testing')

train_generator, validation_generator, test_generator = train_val_generators(training_dir, validation_dir, testing_dir)

Found 3009 images belonging to 2 classes.
Found 376 images belonging to 2 classes.
Found 377 images belonging to 2 classes.


### find accurracy

In [ ]:
loss, accuracy = my_model.evaluate(test_generator)
print('Test accuracy :', accuracy)

12/12 [==============================] - 35s 3s/step - loss: 0.3178 - accuracy: 0.8859
Test accuracy : 0.8859416246414185


In [ ]:
my_model.save('/content/drive/MyDrive/Brain/fine_tune_models')

### save as h5 file

In [ ]:
my_model.save('/content/drive/MyDrive/Brain/my_models/model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### load h5 file

In [ ]:
my_mode_l = tf.keras.models.load_model('/content/drive/MyDrive/Brain/my_models/model1.h5')

In [ ]:
loss, accuracy = my_mode_l.evaluate(test_generator)
print('Test accuracy :', accuracy)

12/12 [==============================] - 11s 652ms/step - loss: 0.3178 - accuracy: 0.8859
Test accuracy : 0.8859416246414185


# evaluating Precision, Recall , specificity and F1 score

### get testing images

In [ ]:
selected_paths = [] ; selected_labels = []
testing_path_0 = os.path.join(base_dir, 'testing', '0')
for x in random.sample(os.listdir(testing_path_0),10):
    selected_paths.append(os.path.join(testing_path_0, x))
    selected_labels.append(0)

testing_path_1 = os.path.join(base_dir, 'testing', '1')
for x in random.sample(os.listdir(testing_path_1),10):
    selected_paths.append(os.path.join(testing_path_1, x))
    selected_labels.append(1)

In [ ]:
X, y = shuffle(selected_paths, selected_labels, random_state=0)

### get predictions

In [ ]:
y_truee = []
y_pred = []
for vis_image, y_true in zip(X, y):
    img = image.load_img(vis_image, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 127.5
    x = np.expand_dims(x, axis=0)
    preprocess_images = np.vstack([x])
    classes = model.predict(preprocess_images, batch_size=10)
    score = tf.nn.sigmoid(classes[0])
    if classes[0]>0.5:
        predicted_label = 1
    else:
        predicted_label =0
    y_truee = y_truee.append(y_true)
    y_pred = y_pred.append(predicted_label)
    #plt.title(f'True Label: {y_true} \n Predicted Label: {predicted_label} with a {100 * np.max(score):.2f} percent confidence.')
    #plt.imshow(img)
    #plt.show()